<a href="https://colab.research.google.com/github/rizkiar00/Bagging-Berbasis-Naive-Bayes/blob/master/Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import

In [560]:
!git clone https://github.com/rizkiar00/Bagging-Berbasis-Naive-Bayes

fatal: destination path 'Bagging-Berbasis-Naive-Bayes' already exists and is not an empty directory.


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
%matplotlib inline

###AMBIL DATA TRAIN

In [562]:
train = np.genfromtxt('Bagging-Berbasis-Naive-Bayes/TrainsetTugas4ML.csv', delimiter=',', skip_header=1)
dftrain = pd.DataFrame(data=train)
print("Kelas:", pd.Series(dftrain.loc[:,2], name='A').unique())
dftrain[2].value_counts()

Kelas: [2. 1.]


1.0    220
2.0     78
Name: 2, dtype: int64

###LAKUKAN TRAINING

*   JUMLAH MODEL DAN JUMLAH DATA YANG DIPAKAI TRAINING BEBAS
*   LAKUKAN TESTING KE SISA DATA YANG BELUM DIGUNAKAN UNTUK TRAINING
*   BAGGING MELAKUKAN VOTING UNTUK TIAP MODEL, KELAS MANAKAH SUATU DATA MASUK
*   LIHAT AKURASINYA




In [563]:
jumlahmodel = 10
array = np.arange(0,dftrain.shape[0])
random = np.random.randint(dftrain.shape[0], size=(jumlahmodel,75))
test = [i for i in array if i not in random]
print("Jumlah Sisa: ",len(test))
Xtrainrandom = []
Ytrainrandom = []
for i in range(jumlahmodel):
  Ytrainrandom.append(np.delete(train[random[i]], (0,1), axis=1)) 
  Xtrainrandom.append(np.delete(train[random[i]], 2, axis=1)) 

objmodel = list()
for i in range(jumlahmodel):
    objmodel.append(GaussianNB().fit(Xtrainrandom[i],Ytrainrandom[i].ravel()))

Ytestsisa = np.delete(train[test], (0,1), axis=1)
Xtestsisa = np.delete(train[test], 2, axis=1)
Ytestpredict = []
for i in range(jumlahmodel):
  Ytestpredict.append(objmodel[i].predict(Xtestsisa))
  print("Akurasi Model {}: {}".format(i,np.mean(Ytestpredict[i] == Ytestsisa)))
  
BaggingYtestpredict = []
for i in range(len(Ytestsisa)):
  votebagging = np.array(Ytestpredict)[:,i]
  votebagging = votebagging.astype(int)
  counts = np.bincount(votebagging)
  BaggingYtestpredict.append(np.argmax(counts))
print("Akurasi Model Bagging: {}".format(np.mean(BaggingYtestpredict == np.transpose(Ytestsisa))))

Jumlah Sisa:  24
Akurasi Model 0: 0.5868055555555556
Akurasi Model 1: 0.6041666666666666
Akurasi Model 2: 0.5868055555555556
Akurasi Model 3: 0.5868055555555556
Akurasi Model 4: 0.5868055555555556
Akurasi Model 5: 0.6041666666666666
Akurasi Model 6: 0.5868055555555556
Akurasi Model 7: 0.5868055555555556
Akurasi Model 8: 0.6041666666666666
Akurasi Model 9: 0.5694444444444444
Akurasi Model Bagging: 1.0


In [564]:
confusion_df = pd.DataFrame(confusion_matrix(Ytestsisa,BaggingYtestpredict),
             columns=["Predicted Class " + str(class_name) for class_name in [1.0,2.0]],
             index = ["Class " + str(class_name) for class_name in [1.0,2.0]])
print(confusion_df)

           Predicted Class 1.0  Predicted Class 2.0
Class 1.0                   17                    0
Class 2.0                    0                    7


###LAKUKAN TESTING UNTUK TESTSETTUGAS4ML.CSV

In [567]:
datatest = np.genfromtxt('Bagging-Berbasis-Naive-Bayes/TestsetTugas4ML.csv', delimiter=',', skip_header=1,usecols = (0,1))
datatest

array([[ 3.3 , 15.45],
       [ 3.9 , 21.85],
       [ 4.6 , 18.25],
       [ 5.2 , 23.4 ],
       [ 7.05, 19.9 ],
       [ 9.75, 25.5 ],
       [11.65, 26.85],
       [12.75, 22.05],
       [14.05, 26.55],
       [14.15, 17.35],
       [14.2 , 22.15],
       [15.2 , 11.7 ],
       [15.95, 16.45],
       [16.05, 11.9 ],
       [16.5 , 15.15],
       [16.95,  9.7 ],
       [17.1 , 14.  ],
       [17.25, 13.15],
       [17.3 , 14.9 ],
       [17.55,  8.1 ],
       [17.55, 25.2 ],
       [18.05, 11.9 ],
       [18.25, 20.2 ],
       [18.4 , 10.05],
       [18.65,  6.9 ],
       [19.05,  8.85],
       [19.95,  8.3 ],
       [20.1 , 21.6 ],
       [20.45,  5.15],
       [20.95, 18.25],
       [21.3 , 22.3 ],
       [21.4 ,  8.8 ],
       [21.4 , 25.85],
       [21.85,  8.5 ],
       [22.15,  5.05],
       [22.4 , 16.5 ],
       [22.6 ,  6.7 ],
       [22.95,  4.75],
       [23.2 ,  5.3 ],
       [23.45,  3.6 ],
       [24.55,  4.  ],
       [25.15, 19.8 ],
       [25.2 ,  6.55],
       [25.

In [568]:
Ydatatestpredict = []
for i in range(jumlahmodel):
  Ydatatestpredict.append(objmodel[i].predict(datatest))

BaggingDataTestpredict = []
for i in range(len(datatest)):
  votebagging = np.array(Ydatatestpredict)[:,i]
  votebagging = votebagging.astype(int)
  counts = np.bincount(votebagging)
  BaggingDataTestpredict.append(np.argmax(counts))
BaggingDataTestpredict

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]